<a href="https://colab.research.google.com/github/TheAmirHK/Genetic_Algorithm_Parallelization/blob/main/NSGA3_Codes/NSGA3(not_completed).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import random
from tqdm import tqdm
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
# In[define the parameters and variables]
INTEGER_INDICES = [0, 1]
CONTINUOUS_INDICES = [2, 3]
BOUNDS = [(1, 10)] * 4

pop_size = 500
generations = 500
MUTATION_RATE = 0.15
CROSSOVER_RATE = 0.92

In [ ]:
# In[define the objective functions]
def evaluate(individual):
    x1, x2, x3, x4 = individual
    obj1 = x1**2 + 2*x2**2 - 4*x3**2 + 11*x4**3  # minimize
    obj2 = -(x1 - 2*x2**3 - 10*x3**2 - 11*x4**2 + x1*x2)  # maximize
    obj3 = x1*x2 + x3**2 - 5*x4  # minimize
    return obj1, obj2, obj3

In [ ]:
# In[define the constraints]
def _constraints(individual):
    x1, x2, x3, x4 = individual
    return (x1 + x2 <= 5) and (x3**2 + x4**2 - x2*x1*x3 <= 20)

In [ ]:
# In[check if fitness values fit1 dominate fit2]
def dominates(fit1, fit2):
    return all(x <= y for x, y in zip(fit1, fit2)) and any(x < y for x, y in zip(fit1, fit2))

In [ ]:
# In[mutation function]
def mutate(individual):
    for i in range(len(individual)):
        if random.random() < MUTATION_RATE:
            if i in INTEGER_INDICES:
                individual[i] = random.randint(*BOUNDS[i])
            else:
                individual[i] += random.uniform(-0.5, 0.5)
                individual[i] = max(BOUNDS[i][0], min(BOUNDS[i][1], individual[i]))
    return individual

In [ ]:
# In[crossover function]
def crossover(parent1, parent2):
    if random.random() < CROSSOVER_RATE:
        point = random.randint(1, len(parent1) - 1)
        child1 = parent1[:point] + parent2[point:]
        child2 = parent2[:point] + parent1[point:]

        for i in INTEGER_INDICES:
            child1[i] = int(round(child1[i]))
            child2[i] = int(round(child2[i]))

        return child1, child2
    return parent1, parent2

In [ ]:
# In[non-dominated sorting]
def non_dominated_sort(population, fitness_values):
    fronts = [[]]
    dom_count = {}
    dom_set = {}

    for i, fit in enumerate(fitness_values):
        dom_set[i] = []
        dom_count[i] = 0
        for j, other_fit in enumerate(fitness_values):
            if dominates(fit, other_fit):
                dom_set[i].append(j)
            elif dominates(other_fit, fit):
                dom_count[i] += 1
        if dom_count[i] == 0:
            fronts[0].append(i)

    current_front = 0
    while fronts[current_front]:
        next_front = []
        for i in fronts[current_front]:
            for j in dom_set[i]:
                dom_count[j] -= 1
                if dom_count[j] == 0:
                    next_front.append(j)
        current_front += 1
        fronts.append(next_front)
    return fronts[:-1]

In [ ]:
# In[reference point generation]
def generate_reference_points(num_points, num_objectives):
    return np.random.dirichlet(np.ones(num_objectives), num_points)

In [ ]:
# I have to write the niching selection